# Web Scaping

In [6]:
import urllib.request as request
import json
import requests

query = "fuel cell materials".replace(" ", '%20')
# api_key = "eb1c6904773529c49823f3fa8320d20b"
api_key = '78c1b783fb8039d53a5fc140809f974f' # me
# api_key = 'e8beab4e5dbb6234709a2f9568f6f036'

    
def find_dois(year, start, max_return = 100):
    global query, api_key

    url = "http://api.springernature.com/meta/v2/json?&q={}%20type:Journal%20year:{}&s={}&p={}&api_key={}".format(query, year, start, max_return, api_key) 
    try:
        crawl_content = request.urlopen(url).read()
        Content = json.loads(crawl_content.decode('utf8'))
        Dois = []
        for i in range(max_return):
            try:
                Dois.append(Content['records'][i]['doi'])
            except:
                continue
        
        # print(Dois)
        return Dois
    except Exception as e:
        return []
    
def find_xml(papers):
    global api_key

    DoiUrls = []
    for doi in papers:
        doiUrls = "https://api.springernature.com/meta/v2/pam?q=doi:{}&p=2&api_key={}".format(doi,api_key)
        DoiUrls.append(doiUrls)
    return DoiUrls
    
    

In [ ]:
dois = []

In [ ]:
with open('year.txt') as f:
    year = int(f.read().strip())

if year > 1980:
    result = []
    print(year)
    for i in range(51):
        result += find_dois(year, i)
        print(len(dois) + len(result), end = ' ')
    print()
    dois += result
    with open('year.txt', 'w') as f:
        f.write(str(year - 1))

with open('springer_dois.json', 'a') as f:
    for doi in dois:
        f.write(doi)
        f.write('\n')



In [2]:
with open('springer_dois.json') as f:
    dois = list(set(f.read().strip().split('\n')))
    print(len(dois))
    


64535


In [ ]:
with open('springer_dois.json', 'w') as f:
    for doi in dois:
        f.write(doi)
        f.write('\n')

In [8]:
xmls = find_xml(dois)
with open('start.txt') as f: start = int(f.read().strip())

for i in range(start, len(xmls)):
    try:
        content = request.urlopen(xmls[i]).read()
        with open(r'springer_papers/springer_{}.xml'.format(i),'wb') as f:
            if content is not None:2
                f.write(content)
                if i % 50 == 0: print(i, end = ' ')
                with open('start.txt', 'w') as f: f.write(str(i))
    except Exception:
        content = None
        continue


60250 60300 60350 60400 60500 60550 60600 60650 60700 60750 60800 60850 60900 60950 61000 61050 61100 61150 61200 61250 61300 61350 61400 61450 61500 61550 61600 61650 61700 61750 61800 61850 61900 61950 62000 62050 62100 62150 62200 62250 62300 62350 62400 62450 62500 62550 62600 62650 62700 62750 62800 62850 62900 62950 63000 63050 63100 63150 63200 63250 63300 63350 63400 63450 63500 63550 63600 63650 63700 63750 63800 63850 63900 

# Tokenizing

In [1]:
from bs4 import BeautifulSoup

with open('springer_dois.json') as f:
    dois = f.read().strip().split('\n')
    print(len(dois))

# NOTE: 提取 DOI, abstract, title, 和 year
def extract(folder, xml): 
    f = open(folder + '/' + xml, encoding = 'utf-8').read()
    doc = BeautifulSoup(f, 'xml')
    if doc.find('doi') is None: return
    p_doi = doc.find('query').string[4:]
    p_abst = None
    p_title = None
    p_year = None

    if doc.find('title') != None and doc.find('title').string.lower() != 'abstract':
        temp = doc.find('title').string.split()
        p_title = ' '.join(temp)

    if doc.find('publicationDate') != None:
        p_year = int(doc.find('publicationDate').string[:4])

    if doc.find('h1') != None and doc.find('h1').string.lower() == 'abstract': 
        temp = doc.find('h1').next_sibling.string.split()
        p_abst = ' '.join(temp)
    elif doc.find('p') != None and doc.find('p').string.lower() == 'abstract':
        temp = doc.find('p').next_sibling.string.split()
        p_abst = ' '.join(temp)
    elif doc.find('title') != None and doc.find('title').string.lower() == 'abstract':
        temp = doc.find('title').next_sibling.string.split()
        p_abst = ' '.join(temp)

    return (p_doi, p_abst, p_title, p_year)



64535


In [2]:
# NOTE: 将论文装进字典里
doi_absts = {}
index = 0
for index in range(0, len(dois)):
    try:
        name = f'springer_{index}.xml'
        result = extract('springer_papers', name)
        if result is None: continue
        else: doi, abst, title, year = result
        if year not in doi_absts: doi_absts[year] = [(doi, title, abst)]
        else: doi_absts[year].append((doi, title, abst))
        if index % 200 == 0: print(index, end = ' ')
    except Exception: continue

print(index)

0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800 3000 3200 3400 3600 3800 4000 4200 4400 4600 4800 5000 5200 5400 5600 5800 6000 6200 6400 6600 6800 7000 7200 7400 7600 7800 8000 8200 8400 8600 8800 9000 9200 9400 9600 9800 10000 10200 10400 10600 10800 11000 11200 11400 11600 11800 12000 12200 12400 12600 12800 13000 13200 13400 13600 13800 14000 14200 14400 14600 14800 15000 15200 15400 15600 15800 16000 16200 16400 16600 16800 17000 17200 17400 17600 17800 18000 18200 18400 18600 18800 19000 19200 19400 19600 19800 20000 20200 20400 20600 20800 21000 21200 21400 21600 21800 22000 22200 22400 22600 22800 23000 23200 23400 23600 23800 24000 24200 24400 24600 24800 26400 26600 26800 27000 27200 27400 27600 27800 28000 28200 28400 28600 28800 29000 29200 29400 29600 29800 30000 30200 30400 30600 30800 31000 31200 31400 31600 31800 32000 32200 32400 32600 32800 33000 33200 33400 33600 33800 34000 34200 34400 34600 34800 35000 35200 35400 35600 35800 36000 36200 36400 3

In [12]:
from pprint import pprint
print(len(doi_absts))
for i in doi_absts: print(i, len(doi_absts[i]))
# pprint(doi_absts)

21
2014 4241
2003 448
2015 4190
2017 4198
2021 4161
2019 4326
2011 2863
2012 3524
2009 1873
2006 922
2013 4303
2020 4277
2010 2141
2018 4325
2008 1742
2005 617
2016 4389
2007 1257
2002 166
2004 534
2001 192


In [8]:
# 创建数据库
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_papers = client['papers']

# 初始化数据库
def papers_init(db_papers):
    # for col in db_papers.list_collection_names():
    #     de = db_papers[col]
    #     db_papers.drop_collection(de)

    for year in range(2001, 2022):
        col = db_papers[str(year)]
        for i in doi_absts[year]:
            doi, title, abst = i
            paper = {
                'doi': doi,
                'title': title,
                'abst': abst
            }
            col.insert_one(paper)
papers_init(db_papers)

In [9]:
# 从数据库中读论文
year_papers = {}
# print(db_papers.list_collection_names())
for year in db_papers.list_collection_names():
    for paper in db_papers[year].find({}, {'_id': 0}):
        if year not in year_papers: year_papers[year] = [(paper['doi'], paper['title'], paper['abst'])]
        else: year_papers[year].append((paper['doi'], paper['title'], paper['abst']))
# print(year_papers)

In [10]:
# 一共有多少单词
text = []
for year in year_papers:
    for paper in year_papers[year]:
        abst = paper[2]
        
        # print(doi_absts[doi])
        if abst != None : text += abst.strip().split(' ')
print(len(text))
print(len(year_papers))
# print(text)

17580778
22


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
all_tokens = {}
# freq = {} # 计算每个词出现的频率
# for doi, (abst, title, year) in doi_absts.items():
for year, paper_lst in year_papers.items():
    # print(year_papers.items())
    
    index = 0
    print()
    print(year)
    for paper in paper_lst:
        doi, title, abst = paper
    # for doi, title, abst in paper_lst:
        if index % 500 == 0: print(index, end = ' ')
        index += 1

        doc = nlp(str(abst))
        # print(doc)
        tokens = []
        for token in doc:
            if token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and not token.is_stop:
                if token.pos_ == 'NUM':
                    tokens.append('#NUM')
                    # if '#NUM' not in freq:
                    #     freq['#NUM'] = 1
                    # else: freq['#NUM'] += 1
                else: 
                    tokens.append(token.lemma_)
                    # if token.lemma_ not in freq: freq[token.lemma_] = 1
                    # else: freq[token.lemma_] += 1

        if year not in all_tokens: all_tokens[year] = [(doi, title, tokens)]
        else: all_tokens[year].append((doi, title, tokens))
    

print(len(all_tokens))
# TODO 去掉频率很低的词(目前论文数量太少，先不弄)
        


In [ ]:
print(all_tokens['2020'])

In [ ]:
# 初始化 db_tokens
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db_tokens = client['s_tokens']
def tokens_init(db_tokens):
    for col in db_tokens.list_collection_names():
        de = db_tokens[col]
        db_tokens.drop_collection(de)

    for col in db_tokens.list_collection_names():
        de = db_tokens[col]
        db_tokens.drop_collection(de)
    for year in range(2000, 2022):
        col = db_tokens[str(year)]
        for i in all_tokens[str(year)]:
            doi, title, tokens = i
            paper = {
                'doi': doi,
                'title': title,
                'tokens': tokens
            }
            col.insert_one(paper)


In [ ]:
# 从数据库中取 all_tokens
all_tokens = {}
for year in db_tokens.list_collection_names():
    for paper in db_tokens[year].find({}, {'_id': 0}):
        if year not in all_tokens: all_tokens[year] = [(paper['doi'], paper['title'], paper['tokens'])]
        else: all_tokens[year].append((paper['doi'], paper['title'], paper['tokens']))

# print(all_tokens)

In [ ]:
# 建立语料库
from gensim import utils

class Corpus:
    def __iter__(self):
        for year in all_tokens:
            token_lst = all_tokens[year]
            for tup in token_lst:
                tokens = tup[-1]
                yield tokens

corpus = Corpus()


In [ ]:
from gensim import corpora

all_texts = []
for doi, tokens in all_tokens.items():
    all_texts.append(tokens)
# print(all_texts)
dic = corpora.Dictionary(all_texts)
# print(all_texts)
print(dic.token2id)
